# ETL Pipeline: Download Historical Open Interest Data

## 📊 Overview
This notebook implements an ETL (Extract, Transform, Load) pipeline for downloading historical open interest (OI) data from cryptocurrency exchanges. It handles rate limiting, batch processing, and data caching for efficient data collection.

## 🎯 Objectives
1. **Data Extraction**: Download historical open interest data from exchange APIs
2. **Batch Processing**: Handle multiple trading pairs and timeframes efficiently
3. **Rate Limit Management**: Respect exchange API limits to avoid throttling
4. **Data Caching**: Store downloaded data locally for future use
5. **Visualization**: Preview downloaded data with interactive charts

## 📋 Prerequisites
- Exchange API access (e.g., Binance Perpetual)
- Network connection for API calls
- Sufficient disk space for data storage (~50MB per pair/interval)

## ⚠️ Important Notes on Rate Limiting
Open Interest API endpoints typically have lower rate limits:
- **Batch Size**: Keep `BATCH_OI_REQUEST` very low (2-5) to avoid hitting limits
- **Sleep Time**: Use longer `SLEEP_REQUEST` (5+ seconds) for safety
- **Monitor**: Watch for 429 (Too Many Requests) errors and adjust accordingly

## 📈 Expected Outputs
- Cached OI data in `app/data/cache/oi/`
- Hourly interval data (1h is recommended for OI)
- Interactive charts showing OI trends for data validation

In [1]:
from core.data_sources.clob import CLOBDataSource
import warnings
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

# Main class to access central limit order book connectors
clob = CLOBDataSource()

# Open Interest config
CONNECTOR_NAME = "binance_perpetual"
INTERVAL = "1h"  # 1h is recommended for OI data

DAYS = 7  # Number of days of historical data to download

# Rate limits config (more conservative for OI)
BATCH_OI_REQUEST = 20  # Number of trading pairs to request in each batch
SLEEP_REQUEST = 5  # Seconds to wait between batches

In [2]:
# Get trading rules and pairs
trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.get_all_trading_pairs()

print(f"Found {len(trading_pairs)} trading pairs")
print(f"First 10 pairs: {trading_pairs[:10]}")

# Since all Binance perpetual pairs support OI data, download all pairs
# You can limit this for faster testing by changing trading_pairs to trading_pairs[:50]
selected_pairs = trading_pairs  # Use all pairs - change to [:50] for testing
print(f"Downloading OI data for {len(selected_pairs)} pairs")

# Download OI data for all pairs
print(f"\nDownloading {INTERVAL} OI data for last {DAYS} days...")
print(f"Batch size: {BATCH_OI_REQUEST}, Sleep time: {SLEEP_REQUEST}s")

all_oi_data = await clob.get_oi_batch_last_days(
    CONNECTOR_NAME, 
    selected_pairs,
    INTERVAL, 
    DAYS, 
    BATCH_OI_REQUEST,
    SLEEP_REQUEST,
)

print(f"\nCompleted downloading OI data for {len(all_oi_data)} trading pairs")

Found 535 trading pairs
First 10 pairs: ['STEEM-USDT', 'SYN-USDT', 'LRC-USDT', '1MBABYDOGE-USDT', 'BIO-USDC', 'FLOCK-USDT', 'CTSI-USDT', 'TRUMP-USDC', '1000BONK-USDT', 'OXT-USDT']

Batch size: 20, Sleep time: 5s
OI Batch 1/27
Start: 0, End: 20
OI Batch 2/27
Start: 20, End: 40
OI Batch 3/27
Start: 40, End: 60
OI Batch 4/27
Start: 60, End: 80
OI Batch 5/27
Start: 80, End: 100
OI Batch 6/27
Start: 100, End: 120
OI Batch 7/27
Start: 120, End: 140
OI Batch 8/27
Start: 140, End: 160
OI Batch 9/27
Start: 160, End: 180
OI Batch 10/27
Start: 180, End: 200
OI Batch 11/27
Start: 200, End: 220
OI Batch 12/27
Start: 220, End: 240
OI Batch 13/27
Start: 240, End: 260
OI Batch 14/27
Start: 260, End: 280
OI Batch 15/27
Start: 280, End: 300
OI Batch 16/27
Start: 300, End: 320
OI Batch 17/27
Start: 320, End: 340
OI Batch 18/27
Start: 340, End: 360
OI Batch 19/27
Start: 360, End: 380
OI Batch 20/27
Start: 380, End: 400
OI Batch 21/27
Start: 400, End: 420
OI Batch 22/27
Start: 420, End: 440
OI Batch 23/27


In [6]:
# Display info about downloaded OI data
successful_downloads = [oi_df for oi_df in all_oi_data if not oi_df.empty]
failed_downloads = len(all_oi_data) - len(successful_downloads)

print(f"Successfully downloaded: {len(successful_downloads)} pairs")
print(f"Failed downloads: {failed_downloads} pairs")

if successful_downloads:
    print("\nSuccessful downloads:")
    for i, oi_df in enumerate(successful_downloads[:10]):  # Show first 10
        pair_name = selected_pairs[i]  # Use selected_pairs list
        print(f"{i+1}. {pair_name}: {len(oi_df)} records")
        
    # Show sample data structure
    print(f"\nSample data structure (first pair):")
    sample_df = successful_downloads[0]
    print(f"Columns: {list(sample_df.columns)}")
    print(f"Date range: {sample_df.index.min()} to {sample_df.index.max()}")
    print("\nFirst 5 records:")
    print(sample_df.head())
else:
    print("No successful downloads to display")
    print("\nDebugging: Let's check what's in the all_oi_data list:")
    for i, oi_df in enumerate(all_oi_data[:10]):  # Show first 10 for debugging
        if i < len(selected_pairs):
            print(f"  Pair {selected_pairs[i]}: DataFrame shape = {oi_df.shape}, empty = {oi_df.empty}")

Successfully downloaded: 535 pairs
Failed downloads: 0 pairs

Successful downloads:
1. STEEM-USDT: 168 records
2. SYN-USDT: 168 records
3. LRC-USDT: 168 records
4. 1MBABYDOGE-USDT: 168 records
5. BIO-USDC: 168 records
6. FLOCK-USDT: 168 records
7. CTSI-USDT: 168 records
8. TRUMP-USDC: 168 records
9. 1000BONK-USDT: 168 records
10. OXT-USDT: 168 records

Sample data structure (first pair):
Columns: ['symbol', 'sumOpenInterest', 'sumOpenInterestValue', 'CMCCirculatingSupply']
Date range: 2025-09-12 15:00:00 to 2025-09-19 14:00:00

First 5 records:
                        symbol  sumOpenInterest  sumOpenInterestValue  \
timestamp                                                               
2025-09-12 15:00:00  STEEMUSDT         13272004      1779249.76688148   
2025-09-12 16:00:00  STEEMUSDT         13267988         1775654.83404   
2025-09-12 17:00:00  STEEMUSDT         13262664         1784624.06784   
2025-09-12 18:00:00  STEEMUSDT         13213184      1794554.92728832   
2025-09-12 

In [7]:
# Visualize OI data for all successful pairs
if successful_downloads:
    # Get all pairs with valid OI data
    valid_pairs = []
    
    for i, oi_df in enumerate(successful_downloads):
        if not oi_df.empty and 'sumOpenInterestValue' in oi_df.columns:
            avg_oi_value = oi_df['sumOpenInterestValue'].mean()
            pair_name = selected_pairs[i]  # Use selected_pairs list
            valid_pairs.append((pair_name, avg_oi_value, oi_df))
    
    # Sort by average OI value (highest first)
    valid_pairs.sort(key=lambda x: x[1], reverse=True)
    
    if valid_pairs:
        # Limit visualization to top N pairs for readability
        max_pairs_to_plot = 40  # Show top 40 pairs (10 rows x 4 cols)
        pairs_to_plot = valid_pairs[:max_pairs_to_plot]
        
        # Calculate subplot layout: 4 columns, n rows
        cols = 4
        rows = (len(pairs_to_plot) + cols - 1) // cols  # Ceiling division
        
        # Calculate appropriate vertical spacing based on number of rows
        if rows <= 5:
            vertical_spacing = 0.04
        elif rows <= 10:
            vertical_spacing = 0.03
        else:
            vertical_spacing = 0.02
        
        # Create subplots with 4 columns
        fig = make_subplots(
            rows=rows, 
            cols=cols,
            subplot_titles=[f"{pair}<br>${value/1e6:.1f}M" for pair, value, _ in pairs_to_plot],
            vertical_spacing=vertical_spacing,
            horizontal_spacing=0.05
        )
        
        for idx, (pair_name, avg_value, oi_df) in enumerate(pairs_to_plot):
            row = (idx // cols) + 1
            col = (idx % cols) + 1
            
            # Create hover template with 2 decimal places
            hover_template = (
                f"{pair_name}<br>" +
                "Time: %{x}<br>" +
                "OI Value: $%{customdata:.2f}M<br>" +
                "<extra></extra>"
            )
            
            # Plot OI Value
            fig.add_trace(
                go.Scatter(
                    x=oi_df.index,
                    y=oi_df['sumOpenInterestValue'] / 1e6,  # Convert to millions for display
                    customdata=oi_df['sumOpenInterestValue'] / 1e6,  # For hover template
                    name=f"{pair_name}",
                    line=dict(width=2, color='#1f77b4'),
                    showlegend=False,
                    hovertemplate=hover_template
                ),
                row=row, col=col
            )
            
            # Update axes for each subplot
            fig.update_xaxes(showticklabels=False, row=row, col=col)
            fig.update_yaxes(
                showticklabels=True, 
                row=row, col=col, 
                tickformat='.1f',  # Show 1 decimal place
                title_text="OI ($M)" if col == 1 else ""  # Only show y-axis title on leftmost column
            )
        
        # Update overall layout
        fig.update_layout(
            height=400 * rows,  # Increased height per row
            title_text=f"Top {len(pairs_to_plot)} Trading Pairs by Open Interest Value ({INTERVAL} interval)",
            showlegend=False,
            margin=dict(l=60, r=30, t=100, b=50),
            font=dict(size=10)
        )
        
        fig.show()
        
        # Display summary statistics for ALL pairs
        print(f"\n📊 OPEN INTEREST SUMMARY ({len(valid_pairs)} total pairs):")
        print("=" * 80)
        
        # Show top 20 pairs statistics
        print("\nTop 20 Pairs by Average OI Value:")
        print("-" * 80)
        for i, (pair_name, avg_value, oi_df) in enumerate(valid_pairs[:20], 1):
            print(f"{i:2d}. {pair_name:15s} | Avg: ${avg_value/1e6:8.2f}M | Max: ${oi_df['sumOpenInterestValue'].max()/1e6:8.2f}M | Points: {len(oi_df):3d}")
        
        # Overall statistics
        print("\n" + "=" * 80)
        total_oi_value = sum(pair[1] for pair in valid_pairs)
        print(f"Total Average OI Value across all pairs: ${total_oi_value/1e9:.2f}B")
        print(f"Average OI Value per pair: ${total_oi_value/len(valid_pairs)/1e6:.2f}M")
        
    else:
        print("No valid OI data found for visualization")
else:
    print("No data available for visualization")


📊 OPEN INTEREST SUMMARY (535 total pairs):

Top 20 Pairs by Average OI Value:
--------------------------------------------------------------------------------
 1. BTC-USDT        | Avg: $10400.82M | Max: $10804.03M | Points: 168
 2. ETH-USDT        | Avg: $ 9071.77M | Max: $ 9973.53M | Points: 168
 3. SOL-USDT        | Avg: $ 2980.66M | Max: $ 3219.53M | Points: 168
 4. ETH-USDC        | Avg: $ 1406.27M | Max: $ 1470.09M | Points: 168
 5. XRP-USDT        | Avg: $ 1045.96M | Max: $ 1110.26M | Points: 168
 6. BTC-USDC        | Avg: $ 1017.50M | Max: $ 1065.11M | Points: 168
 7. DOGE-USDT       | Avg: $  897.73M | Max: $ 1013.97M | Points: 168
 8. BNB-USDT        | Avg: $  694.94M | Max: $  812.39M | Points: 168
 9. ENA-USDT        | Avg: $  396.11M | Max: $  442.41M | Points: 168
10. SUI-USDT        | Avg: $  339.02M | Max: $  386.39M | Points: 168
11. WLFI-USDT       | Avg: $  323.55M | Max: $  364.65M | Points: 168
12. AVAX-USDT       | Avg: $  291.74M | Max: $  381.68M | Points: 168


In [8]:
# Check OI feed cache status
try:
    # Try to get the OI feed (lazy-loaded)
    oi_feed = clob._get_oi_feed(CONNECTOR_NAME)
    cache_status = oi_feed.get_cache_status()
    
    print("📁 OI CACHE STATUS:")
    print(f"   Feed type: {cache_status['feed_type']}")
    print(f"   Memory cache entries: {cache_status['memory_cache_entries']}")
    print(f"   Disk cache files: {cache_status['disk_cache_files']}")
    print(f"   Total disk size: {cache_status['total_disk_size_mb']:.2f} MB")
    print(f"   Cache directory: {cache_status['cache_directory']}")
    
    print(f"\n✅ Successfully cached OI data for future use!")
    print(f"   Next time you request the same data, it will load from cache instantly.")
except ValueError:
    print("❌ OI feed not available for this connector")

📁 OI CACHE STATUS:
   Feed type: binance_perpetual_oi
   Memory cache entries: 535
   Disk cache files: 535
   Total disk size: 4.19 MB
   Cache directory: /Users/dman/Documents/code/quants-lab/app/data/cache/oi

✅ Successfully cached OI data for future use!
   Next time you request the same data, it will load from cache instantly.
